In [1]:
import dopo

In [6]:
import brightway2 as bw
import bw2data as bd
import bw2analyzer as ba

In [7]:
bd.projects.create_project("dopo-try1")
bw.bw2setup()

bio3=bw.Database('biosphere3')
ei39=bw.Database('ecoinvent 3.9.1 cutoff')
ei39SSP2=bw.Database('ei_cutoff_3.9_image_SSP2-RCP19_2050 2024-06-27')

Biosphere database already present!!! No setup is needed


In [28]:
fp_yaml='cement_concrete.yaml'
flt_act=dopo.generate_sets_from_filters(fp_yaml, database=ei39)

In [22]:
ei39SSP2

Brightway2 SQLiteBackend: ei_cutoff_3.9_image_SSP2-RCP19_2050 2024-06-27

In [29]:
flt_act

{'Cement': set(),
 'Concrete': set(),
 'Steel': set(),
 'Electricity production all': set(),
 'Electricity production fossil': set(),
 'Electricity production renewables': set(),
 'Electricity production nuclear': set(),
 'Electricity production biomass': set(),
 'Electricity production biomethane': set(),
 'Electricity production hydro': set(),
 'Electricity production geothermal': set(),
 'Electricity production photovoltaic': set(),
 'Electricity production solar': set(),
 'Electricity production wind': set(),
 'Electricity production wood': set(),
 'Electricity production wave energy': set(),
 'Electricity production coal': set(),
 'Electricity production lignite': set(),
 'Electricity production natural gas': set(),
 'Electricity production oil': set(),
 'Electricity production petroleum': set()}

In [19]:
small_cement=[x for x in sets['Cement']]

In [20]:
small_cement

[]